# 廣韻反切系聯

[返回 GitHub 頁面](https://github.com/sgalal/Guangyun)

In [ ]:
from collections import defaultdict

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
import networkx as nx

In [ ]:
import sqlite3

In [ ]:
conn = sqlite3.connect('../../data.sqlite3')

In [ ]:
cur = conn.cursor()

## 反切上字

In [ ]:
G = nx.Graph()

In [ ]:
G.add_weighted_edges_from(cur.execute('''SELECT 廣韻小韻.上字 AS 上字,
廣韻字頭全.上字 AS 上字的上字,
count(*) AS 權重
FROM 廣韻小韻 INNER JOIN 廣韻字頭全
ON 廣韻小韻.上字 = 廣韻字頭全.字頭
WHERE 廣韻小韻.上字 <> 廣韻字頭全.上字
GROUP BY 廣韻小韻.上字, 廣韻字頭全.上字
ORDER BY 廣韻小韻.小韻號;'''))

In [ ]:
for i in sorted(nx.connected_components(G)):
    print(''.join(i))

# 反切下字

In [ ]:
plt.rcParams['figure.figsize'] = [8, 6]

In [ ]:
def plot(G):
    pos = nx.spring_layout(G, k=4, iterations=100, seed=42)
    labels = nx.get_edge_attributes(G, 'weight')
    nx.draw_networkx_nodes(G, pos, node_size=1000, edgecolors='black', node_color='w')
    nx.draw_networkx_edges(G, pos)
    nx.draw_networkx_labels(G, pos, font_size=18, font_family='Source Han Serif K')
    nx.draw_networkx_edge_labels(G, pos, font_size=12, edge_labels=labels, font_family='Times New Roman')
    plt.show()

In [ ]:
d = defaultdict(list)

In [ ]:
for 韻, 下字, 下字的下字, 權重 in cur.execute(f'''SELECT substr(廣韻小韻.韻, 1, 1),
廣韻小韻.下字 AS 下字,
廣韻字頭全.下字 AS 下字的下字,
count(*) AS 權重
FROM 廣韻小韻 INNER JOIN 廣韻字頭全
ON 廣韻小韻.下字 = 廣韻字頭全.字頭
WHERE 廣韻小韻.下字 <> 廣韻字頭全.下字
AND substr(廣韻小韻.韻, 1, 1) = substr(廣韻字頭全.韻, 1, 1)
GROUP BY 廣韻小韻.下字, 廣韻字頭全.下字
ORDER BY 廣韻小韻.小韻號;'''):
    d[韻].append((下字, 下字的下字, 權重))

In [ ]:
for 韻 in list(d):
    print(韻)
    G = nx.Graph()
    G.add_weighted_edges_from(d[韻])
    plot(G)

In [ ]:
conn.close()